In [2]:
pip install faker

     ---------------------------------------- 1.9/1.9 MB 20.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [3]:
import sqlite3
import random
from faker import Faker

In [4]:
# Create a connection to SQLite database
conn = sqlite3.connect('nvidia_database.db')
cursor = conn.cursor()

In [5]:
# 1. Create tables
cursor.executescript('''
-- Create products table
CREATE TABLE IF NOT EXISTS products (
    product_id INTEGER PRIMARY KEY AUTOINCREMENT,
    product_name TEXT NOT NULL,
    category TEXT NOT NULL,
    release_date DATE,
    price REAL
);

-- Create customers table
CREATE TABLE IF NOT EXISTS customers (
    customer_id INTEGER PRIMARY KEY AUTOINCREMENT,
    customer_name TEXT NOT NULL,
    industry TEXT,
    contact_email TEXT,
    contact_phone TEXT
);

-- Create sales table
CREATE TABLE IF NOT EXISTS sales (
    sale_id INTEGER PRIMARY KEY AUTOINCREMENT,
    product_id INTEGER,
    customer_id INTEGER,
    sale_date DATE,
    region TEXT,
    quantity_sold INTEGER,
    revenue REAL,
    FOREIGN KEY (product_id) REFERENCES products (product_id),
    FOREIGN KEY (customer_id) REFERENCES customers (customer_id)
);

-- Create suppliers table
CREATE TABLE IF NOT EXISTS suppliers (
    supplier_id INTEGER PRIMARY KEY AUTOINCREMENT,
    supplier_name TEXT NOT NULL,
    material_supplied TEXT NOT NULL,
    contact_email TEXT
);

-- Create supply_chain table
CREATE TABLE IF NOT EXISTS supply_chain (
    supply_chain_id INTEGER PRIMARY KEY AUTOINCREMENT,
    supplier_id INTEGER,
    product_id INTEGER,
    supply_date DATE,
    quantity_supplied INTEGER,
    FOREIGN KEY (supplier_id) REFERENCES suppliers (supplier_id),
    FOREIGN KEY (product_id) REFERENCES products (product_id)
);

-- Create departments table
CREATE TABLE IF NOT EXISTS departments (
    department_id INTEGER PRIMARY KEY AUTOINCREMENT,
    department_name TEXT NOT NULL,
    location TEXT
);

-- Create employees table
CREATE TABLE IF NOT EXISTS employees (
    employee_id INTEGER PRIMARY KEY AUTOINCREMENT,
    first_name TEXT NOT NULL,
    last_name TEXT NOT NULL,
    department_id INTEGER,
    hire_date DATE,
    salary REAL,
    FOREIGN KEY (department_id) REFERENCES departments (department_id)
);

-- Create projects table
CREATE TABLE IF NOT EXISTS projects (
    project_id INTEGER PRIMARY KEY AUTOINCREMENT,
    project_name TEXT NOT NULL,
    department_id INTEGER,
    start_date DATE,
    end_date DATE,
    budget REAL,
    FOREIGN KEY (department_id) REFERENCES departments (department_id)
);
''')

# Commit table creation
conn.commit()

In [6]:
# 2. Populate tables with sample data

# Create an instance of Faker for generating random data
fake = Faker()

# Define sample data for each table

# Create 15 products
products_data = [
    ('RTX 4090', 'GPU', '2023-01-15', 1599.99),
    ('RTX 4080', 'GPU', '2022-11-16', 1199.99),
    ('RTX 4070', 'GPU', '2023-03-08', 599.99),
    ('Jetson Nano', 'AI Accelerator', '2022-05-20', 99.99),
    ('Jetson AGX Orin', 'AI Accelerator', '2023-06-10', 1999.99),
    ('DGX Station', 'Server', '2023-08-01', 8999.99),
    ('A100 Tensor Core', 'AI Accelerator', '2021-07-15', 14999.99),
    ('H100 Tensor Core', 'AI Accelerator', '2022-10-10', 19999.99),
    ('GeForce GTX 1660', 'GPU', '2019-02-22', 279.99),
    ('Titan V', 'GPU', '2017-12-07', 2999.99),
    ('GeForce RTX 3060', 'GPU', '2021-10-01', 349.99),
    ('GeForce GTX 1080', 'GPU', '2016-05-27', 499.99),
    ('NVIDIA V100', 'AI Accelerator', '2017-06-10', 8000.00),
    ('NVIDIA A40', 'AI Accelerator', '2020-09-15', 2500.00),
    ('NVIDIA Quadro GV100', 'GPU', '2018-04-15', 8999.00)
]

# Create 20 customers
customers_data = [(fake.company(), fake.random_element(elements=('Gaming', 'Research', 'Data Centers', 'AI', 'Cloud Services')), fake.email(), fake.phone_number()) for _ in range(20)]

# Create 100 sales
sales_data = [(random.randint(1, 15), random.randint(1, 20), fake.date_this_year(), random.choice(['North America', 'Europe', 'Asia', 'Africa']), random.randint(10, 500), round(random.uniform(10000, 200000), 2)) for _ in range(100)]

# Create 50 suppliers
suppliers_data = [(fake.company(), fake.word(), fake.email()) for _ in range(50)]

# Supply chain data should be proportional to sales data (100 sales => 100 supply chain records)
supply_chain_data = [(random.randint(1, 50), random.randint(1, 15), fake.date_this_year(), random.randint(1000, 5000)) for _ in range(100)]

# Create 5 departments (same as before)
departments_data = [
    ('R&D', 'Santa Clara'),
    ('Sales', 'New York'),
    ('Operations', 'Austin'),
    ('Marketing', 'Los Angeles'),
    ('HR', 'Seattle')
]

# Create 30 employees
employees_data = [(fake.first_name(), fake.last_name(), random.randint(1, 5), fake.date_this_decade(), round(random.uniform(50000, 150000), 2)) for _ in range(30)]

# Create 10 projects
projects_data = [(fake.bs(), random.randint(1, 5), fake.date_this_year(), fake.date_this_year(), round(random.uniform(100000, 1000000), 2)) for _ in range(10)]

In [7]:
# 3. Insert data into tables
cursor.executemany('INSERT INTO products (product_name, category, release_date, price) VALUES (?, ?, ?, ?)', products_data)
cursor.executemany('INSERT INTO customers (customer_name, industry, contact_email, contact_phone) VALUES (?, ?, ?, ?)', customers_data)
cursor.executemany('INSERT INTO sales (product_id, customer_id, sale_date, region, quantity_sold, revenue) VALUES (?, ?, ?, ?, ?, ?)', sales_data)
cursor.executemany('INSERT INTO suppliers (supplier_name, material_supplied, contact_email) VALUES (?, ?, ?)', suppliers_data)
cursor.executemany('INSERT INTO supply_chain (supplier_id, product_id, supply_date, quantity_supplied) VALUES (?, ?, ?, ?)', supply_chain_data)
cursor.executemany('INSERT INTO departments (department_name, location) VALUES (?, ?)', departments_data)
cursor.executemany('INSERT INTO employees (first_name, last_name, department_id, hire_date, salary) VALUES (?, ?, ?, ?, ?)', employees_data)
cursor.executemany('INSERT INTO projects (project_name, department_id, start_date, end_date, budget) VALUES (?, ?, ?, ?, ?)', projects_data)

In [8]:
# Commit changes and close the connection
conn.commit()
conn.close()

print("Database populated successfully!")

Database populated successfully!


In [9]:
import os
print(os.getcwd())

C:\Users\imaja\AppData\Local\Programs\Microsoft VS Code
